In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
from model import *
from data import *
from tools import *
from functions import *
import os



In [4]:
## Model parameters

gamma_c = 1
gamma_w = 0.99
eps = 1e-12
nu = 0.5
zeta = 1e-3


# Hyperparameters

T = 10000
K = 2
N = 3

In [5]:
X = torch.load('X.pt')
A = torch.load('A.pt')
Winit = torch.load('Winit.pt')
Cinit = torch.load('Cinit.pt')

X = X.unsqueeze(0)
A = A.unsqueeze(0)
Winit = Winit.unsqueeze(0)
Cinit = Cinit.unsqueeze(0)

In [6]:
X


tensor([[[[ 0.4263, -0.4635],
          [ 0.3313, -0.7141],
          [ 1.8679,  1.0332],
          ...,
          [-0.6193,  0.6471],
          [-2.2981, -1.0233],
          [-0.1628, -0.0034]],

         [[-1.2117,  0.7850],
          [-0.2476, -0.4174],
          [ 1.2633,  0.5610],
          ...,
          [-0.1858, -0.2123],
          [-0.8966,  0.4985],
          [-1.4598, -0.0598]],

         [[-1.7979,  0.2671],
          [ 2.0315,  0.9266],
          [ 1.0584,  0.3782],
          ...,
          [ 0.1198, -0.3894],
          [-0.7366, -1.6368],
          [-0.8747,  0.1703]]]], device='cuda:0')

In [7]:
input_dim = None
N_updates_W = 10
N_updates_C = 1
num_layers = 233
mode = 'end-to-end'
model = myModel(input_dim, N_updates_W, N_updates_C, num_layers=num_layers, gamma_c=gamma_c, gamma_w=gamma_w, eps=eps, nu=nu, zeta=zeta,B=1).cuda()

In [8]:
W_predicted,_ = model(X,A,Winit,Cinit,mode)

s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])
s size:  torch

In [9]:
W_predicted

tensor([[[[-0.5422,  1.2302],
          [-1.2947,  0.6728],
          [ 0.2257, -0.2347]],

         [[-1.1252,  0.4357],
          [ 0.4230, -0.4219],
          [-0.2768,  1.0741]],

         [[-0.1988, -0.4575],
          [ 0.2953,  1.0591],
          [ 1.2524,  0.6034]]]], device='cuda:0', grad_fn=<PermuteBackward0>)

In [10]:
joint_isi(W_predicted[0], A[0])

tensor(0.0385, device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
alpha = torch.ones(1).cuda()

In [30]:
alpha

tensor([1.], device='cuda:0')

In [14]:
coefficients = model.Layers[0].get_coefficients(cov_X(X),alpha,Cinit,Cinit)

In [15]:
c_w,_,_,_ = coefficients

In [19]:
c_w

tensor([0.3007], device='cuda:0')

In [17]:
prox_f(Winit,c_w)

s size:  torch.Size([1, 2, 3])
c_w size:  torch.Size([1])


tensor([[[[-0.1669,  0.9339],
          [-0.8453, -0.3444],
          [-0.7064,  0.6432]],

         [[-1.1260, -0.6793],
          [-0.0342, -1.1951],
          [ 0.1469,  0.2841]],

         [[-0.4657, -0.4355],
          [-0.5456,  1.9536],
          [ 0.7955,  0.2445]]]], device='cuda:0')

In [65]:
A

tensor([[[[ 0.6111,  0.6240],
          [ 0.8011, -0.6099],
          [ 0.1041,  0.4855]],

         [[-0.6772,  0.7226],
          [ 0.4255,  0.6575],
          [ 0.5905, -0.1225]],

         [[ 0.4432, -0.2338],
          [-0.4260,  0.4354],
          [ 0.8176,  0.8540]]]], device='cuda:0')

In [5]:
lambda_1 = 0.04
lambda_2 = 0.25
rho_bounds_1 = [0.2,0.3]
rho_bounds_2 = [0.6,0.7]
rhos = [rho_bounds_1,rho_bounds_2]
lambdas = [lambda_1,lambda_2]

metaparameters_multiparam = get_metaparameters(rhos,lambdas)
metaparameters_titles_multiparam = ['Case A','Case B','Case C','Case D']


X,A = generate_whitened_problem(T,K,N,epsilon=1,rho_bounds=rho_bounds_2,lambda_=lambda_1)
Winit = make_A(K,N)
Cinit = make_Sigma(K,N,rank=K+10)


In [8]:
X = X.unsqueeze(0)
A = A.unsqueeze(0)
Winit = Winit.unsqueeze(0)
Cinit = Cinit.unsqueeze(0)


In [52]:
print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory}")
print(f"Reserved GPU memory: {torch.cuda.memory_reserved()}")
print(f"Allocated GPU memory: {torch.cuda.memory_allocated()}")


Total GPU memory: 34089730048
Reserved GPU memory: 2097152
Allocated GPU memory: 448000


In [55]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("CUDA device count:", torch.cuda.device_count())



PyTorch version: 1.11.0a0+bfe5ad2
CUDA available: True
CUDA version: 11.6
CUDA device count: 1


In [3]:
Cinit
c = 1
eps = 1e-3

In [24]:
C_perm = Cinit.permute(2, 0, 1)
s,U = torch.linalg.eigh(C_perm)
Vh = U.permute(0, 2, 1)
s_new = torch.maximum(torch.tensor(eps), (s + torch.sqrt(s**2 + 2 * c)) / 2)
# s_new = (s + np.sqrt(s**2+2*c_c))/2
diag_s = torch.diag_embed(s_new)  
C_new = torch.einsum('nNv,nvw,nwM -> nNM',U,diag_s,Vh)

In [25]:
C_new

tensor([[[1.3390, 0.3693],
         [0.3693, 1.4140]],

        [[1.4232, 0.4589],
         [0.4589, 1.3981]],

        [[1.3876, 0.5028],
         [0.5028, 1.2744]]], device='cuda:0')

In [33]:
C_perm = Cinit_batch.permute(0, 3, 1, 2)  # (batch, last_dim, height, width)
s, U = torch.linalg.eigh(C_perm)
Vh = U.transpose(-1, -2)
# Calculate the new singular values
s_new = torch.maximum(torch.tensor(eps), (s + torch.sqrt(s**2 + 2 * c)) / 2)
diag_s = torch.diag_embed(s_new)

# Reconstruct the matrix with the new singular values
C_new = torch.einsum('bnNv,bnvw,bnwM -> bnNM',U,diag_s,Vh)

In [35]:
Cinit_batch

tensor([[[[0.9366, 1.0303, 0.9671],
          [0.4744, 0.5879, 0.6687]],

         [[0.4744, 0.5879, 0.6687],
          [1.0329, 0.9981, 0.8166]]]], device='cuda:0')

In [37]:
C_perm

tensor([[[[0.9366, 0.4744],
          [0.4744, 1.0329]],

         [[1.0303, 0.5879],
          [0.5879, 0.9981]],

         [[0.9671, 0.6687],
          [0.6687, 0.8166]]]], device='cuda:0')

In [109]:
sym(C_new)

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

In [40]:
prox_g_batch(Cinit_batch, c, eps)   

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 3

In [41]:
Cinit.shape

torch.Size([2, 2, 3])

In [46]:
Cperm = Cinit_batch.permute(0, 3, 1, 2)

In [67]:
a,B = torch.linalg.eigh(Cinit.permute(2,0,1))

In [100]:
Bh = B.permute(0, 2, 1)
a_new = torch.maximum(torch.tensor(eps), (a + torch.sqrt(a**2 + 2 * c)) / 2)
# s_new = (s + np.sqrt(s**2+2*c_c))/2
diag_s = torch.diag_embed(a_new)  
A_new = torch.einsum('nNv,nvw,nwM -> nNM',B,diag_s,Bh)

In [102]:
A_new.permute(1, 2, 0)

tensor([[[1.3390, 1.4232, 1.3876],
         [0.3693, 0.4589, 0.5028]],

        [[0.3693, 0.4589, 0.5028],
         [1.4140, 1.3981, 1.2744]]], device='cuda:0')

In [69]:
a.shape

torch.Size([3, 2])

In [48]:
s,U = torch.linalg.eigh(Cperm)

In [66]:
s

tensor([[[0.5079, 1.4616],
         [0.4261, 1.6024],
         [0.2189, 1.5648]]], device='cuda:0')

In [76]:
x = (s + torch.sqrt(s**2 + 2 * c)) / 2

In [74]:
torch.tensor(eps)

tensor(0.0010)

In [78]:
x

tensor([[[1.0053, 1.7477],
         [0.9515, 1.8698],
         [0.8250, 1.8370]]], device='cuda:0')

In [83]:
s_new = torch.maximum(x, torch.tensor(eps))

In [85]:
s_new

tensor([[[1.0053, 1.7477],
         [0.9515, 1.8698],
         [0.8250, 1.8370]]], device='cuda:0')

In [90]:
diag_s = torch.diag_embed(s_new)

In [86]:
U

tensor([[[[ 0.7420,  0.6705],
          [-0.6705,  0.7420]],

         [[-0.6974,  0.7167],
          [ 0.7167,  0.6974]],

         [[-0.6664,  0.7456],
          [ 0.7456,  0.6664]]]], device='cuda:0')

In [96]:
C_new = torch.einsum('bnNv,bnvw,bnwM -> bnNM',U,diag_s,U.transpose(-1,-2))


In [98]:
C_new

tensor([[[[1.3390, 0.3693],
          [0.3693, 1.4140]],

         [[1.4232, 0.4589],
          [0.4589, 1.3981]],

         [[1.3876, 0.5028],
          [0.5028, 1.2744]]]], device='cuda:0')

In [97]:
C_new.permute(0, 2, 3, 1)

tensor([[[[1.3390, 1.4232, 1.3876],
          [0.3693, 0.4589, 0.5028]],

         [[0.3693, 0.4589, 0.5028],
          [1.4140, 1.3981, 1.2744]]]], device='cuda:0')

In [26]:
prox_g(Cinit, c, eps)

tensor([[[1.3390, 0.3693],
         [0.3693, 1.4140]],

        [[1.4232, 0.4589],
         [0.4589, 1.3981]],

        [[1.3876, 0.5028],
         [0.5028, 1.2744]]], device='cuda:0')

In [37]:
joint_isi(W_predicted,A)

tensor(0.0486, device='cuda:0', grad_fn=<DivBackward0>)

In [38]:
parameters = list(model.parameters())
print(parameters)

[Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Parameter containing:
tensor(1., device='cuda:0', requires_grad=True), Param

In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [61]:
class ToyNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(2, 512)
    
    def forward(self, x):
        print(x.shape)
        x = self.fc1(x)
        return x

model = ToyNetwork()

In [60]:
nb_examples = 25
X = torch.randn(nb_examples,2)
X_alone = torch.randn(2)

In [64]:
model(X).shape

torch.Size([25, 2])


torch.Size([25, 512])

In [49]:
# Generate a toy dataset
def generate_toy_data(num_samples=1000):
    X = torch.randn(num_samples, 2)
    Y = torch.randn(num_samples, 512)  # Assuming the output should match the dimensions of the layer
    return X, Y

In [55]:

# Define training parameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# Create dataset and dataloader
X, Y = generate_toy_data()
dataset = TensorDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate model, loss function, and optimizer
model = ToyNetwork()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Training loop
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()  # Set the model to training mode
    for batch in dataloader:
        inputs, targets = batch

        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss
        epoch_loss += loss.item()

    # Average loss for the epoch
    avg_loss = epoch_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [1/10], Loss: 1.4671
Epoch [2/10], Loss: 1.4158
Epoch [3/10], Loss: 1.3677
Epoch [4/10], Loss: 1.3296
Epoch [5/10], Loss: 1.2848
Epoch [6/10], Loss: 1.2510
Epoch [7/10], Loss: 1.2209
Epoch [8/10], Loss: 1.1952
Epoch [9/10], Loss: 1.1719
Epoch [10/10], Loss: 1.1469
